In [1]:
import sys, os
PATH = os.path.dirname(os.path.abspath(os.curdir))
if PATH not in sys.path:
    sys.path.insert(0, PATH)

import numpy as np
import scipy as sp
import torch
import h5py

import src.Optimizer as opt
import src.simulation_setup as setup
import hierarchical_algorithm as ha



In [2]:
##* create simulator 

buffer_train = "Experimental_data_CO_O_merged_train.hdf5"

const_dict, sim = setup.create_common_simulator(PATH, data_file=buffer_train)

Data Buffer:  /Users/joseafonso/Desktop/PlasmaDM/Buffer_Data/Experimental_data_CO_O_merged_train.hdf5
  d[CO2_F]/dt = -CO2_F*r_29 + r_28*(-CO2_F - CO_F - O2_F - O_F + 1.0)
  d[CO_F]/dt = -CO_F*O_F*r_35 - 0.02*CO_F*O_S*r_40 - 0.02*CO_F*Odb_S*r_61 - 0.02*CO_F*Vdb_S*r_60 - CO_F*r_31 - CO_F*r_33 - 0.02*CO_F*r_36*(-CO_S - COdb_S - O_S - Odb_S - Vdb_S + 1.0) + r_30*(-CO2_F - CO_F - O2_F - O_F + 1.0)
  d[CO_S]/dt = CO_F*r_36*(-CO_S - COdb_S - O_S - Odb_S - Vdb_S + 1.0) - CO_S*O_F*r_39 - CO_S*r_37 - CO_S*r_43 - CO_S*r_44 - CO_S*r_45 - CO_S*r_46 + r_32*(-CO_S - COdb_S - O_S - Odb_S - Vdb_S + 1.0)
  d[COdb_S]/dt = CO_F*Vdb_S*r_60 - COdb_S*O_F*r_62 - COdb_S*r_54 - COdb_S*r_55 - COdb_S*r_56 - COdb_S*r_57 - COdb_S*r_59 + Vdb_S*r_49
  d[O2_F]/dt = -O2_F*O_F*r_15 - O2_F*r_10 - O2_F*r_12 - O2_F*r_14 + r_9*(-CO2_F - CO_F - O2_F - O_F + 1.0)
  d[O_F]/dt = -CO_F*O_F*r_35 - 0.02*CO_S*O_F*r_39 - 0.02*COdb_S*O_F*r_62 - O2_F*O_F*r_15 - 2*O_F**2*r_8 - 0.02*O_F*O_S*r_7 - 0.02*O_F*Odb_S*r_27 - 0.02*O_F*Vdb_S*r_

In [3]:

# 2. Define Parameters and Bounds
lower_bounds_dict = {
    'A_d': 1e-8, 'B_d': 1e-8, 'E_d': 0.0, 
    'SF_30': 1e-5, 'SF_31': 1e-5, 'SF_32': 1e-5, 'SF_33': 1e-5, 'SF_34': 1e-5, 'SF_35': 1e-5, 'SF_36': 1e-5, 'SF_37': 1e-5, 'SF_38': 1e-5, 'SF_39': 1e-5,
    'SF_49': 1e-5, 'SF_50': 1e-5, 'SF_51': 1e-5, 'SF_52': 1e-5, 'SF_53': 1e-5, 'SF_54': 1e-5, 'SF_55': 1e-5, 'SF_56': 1e-5, 'SF_57': 1e-5, 'SF_58': 1e-5, 'SF_59': 1e-5, 'SF_60': 1e-5, 'SF_61': 1e-5, 'SF_62': 1e-5,
    'Emin': 1.0, 'Ealpha': 2000
}

upper_bounds_dict = {
    'A_d': 5e-1, 'B_d': 1e-2, 'E_d': 30.0, 
    'SF_30': 1.0, 'SF_31': 1.0, 'SF_32': 1.0,  'SF_33': 1.0, 'SF_34': 1.0, 'SF_35': 1.0, 'SF_36': 1.0, 'SF_37': 1.0, 'SF_38': 1.0, 'SF_39': 1.0,
    'SF_49': 1.0, 'SF_50': 1.0, 'SF_51': 1.0, 'SF_52': 1.0, 'SF_53': 1.0, 'SF_54': 1.0, 'SF_55': 1.0, 'SF_56': 1.0, 'SF_57': 1.0, 'SF_58': 1.0, 'SF_59': 1.0, 'SF_60': 1.0, 'SF_61': 1.0, 'SF_62': 1.0,
    'Emin': 5.0, 'Ealpha': 5000
}

params_default_dict = {
    'A_d': 0.02634, 'B_d': 7.67e-4, 'E_d': 10.75, 
    'SF_30': 1.0, 'SF_31': 1.0, 'SF_32': 1e-2,  'SF_33': 1e-1, 'SF_34': 1e-1, 'SF_35': 1e-2, 'SF_36': 1e-1, 'SF_37': 1e-1, 'SF_38': 1e-1, 'SF_39': 1e-1,
    'SF_49': 1e-2, 'SF_50': 1.0, 'SF_51': 1.0, 'SF_52': 1.0, 'SF_53': 1e-1, 'SF_54': 1e-1, 'SF_55': 1.0, 'SF_56': 1.0, 'SF_57': 1.0, 'SF_58': 1e-1, 'SF_59': 1e-1, 'SF_60': 1e-2, 'SF_61': 1e-1, 'SF_62': 1e-1,
    'Emin': 3.4, 'Ealpha': 3000
}

lower_bounds = np.array(list(lower_bounds_dict.values()))
upper_bounds = np.array(list(upper_bounds_dict.values()))
params_default_init = np.array(list(params_default_dict.values()))
params_default_norm = (params_default_init - lower_bounds) * np.reciprocal(upper_bounds - lower_bounds)


def func_optimization(params_input, flag='numpy'):
    
    ##! normalize variables
    params = [0] * len(params_input)
    for idx, param in enumerate(params_input):
        params[idx] = lower_bounds[idx] + (upper_bounds[idx] - lower_bounds[idx]) * param
    
    A_d, B_d, E_d = params[0:3]
    SF_30, SF_31, SF_32, SF_33, SF_34, SF_35, SF_36, SF_37, SF_38, SF_39 = params[3:13]
    SF_49, SF_50, SF_51, SF_52, SF_53, SF_54, SF_55, SF_56, SF_57, SF_58, SF_59, SF_60, SF_61, SF_62 = params[13:27]
    Emin, Ealpha = params[27:]
    
    if flag=='numpy':
        nu_d_mod = lambda T: 1e15 * (A_d + B_d * np.exp(E_d/(const_dict['R'] * T)))
    elif flag=='torch':
        nu_d_mod = lambda T: 1e15 * (A_d + B_d * torch.exp(E_d/(const_dict['R'] * T)))
    else:
        raise ValueError(f"{flag} does not exist")
    
    dict_mod_vec = [
    {"id": 2, "rate": None, "model_dict": {"nu_d": nu_d_mod}},
    {"id": 10, "rate": None, "model_dict": {"nu_d": nu_d_mod}},
    {"id": 16, "rate": None, "model_dict": {"Emin": Emin}},
    {"id": 18, "rate": None, "model_dict": {"Emin": Emin}},
    
    {"id": 31, "rate": None, "model_dict": {"SF": SF_31, "nu_d": nu_d_mod}},
    
    {"id": 30, "rate": None, "model_dict": {"SF": SF_30}},
    {"id": 32, "rate": None, "model_dict": {"SF": SF_32}},
    {"id": 33, "rate": None, "model_dict": {"SF": SF_33}},
    {"id": 34, "rate": None, "model_dict": {"SF": SF_34}},
    
    {"id": 35, "rate": None, "model_dict": {"SF": SF_35}},
    {"id": 36, "rate": None, "model_dict": {"SF": SF_36}},
    {"id": 37, "rate": None, "model_dict": {"SF": SF_37}},
    {"id": 38, "rate": None, "model_dict": {"SF": SF_38}},
    {"id": 39, "rate": None, "model_dict": {"SF": SF_39}},
    
    {"id": 44, "rate": None, "model_dict": {"Emin": Emin}},
    
    {"id": 49, "rate": None, "model_dict": {"SF": SF_49}},
    {"id": 50, "rate": None, "model_dict": {"SF": SF_50, "Ealpha": Ealpha}},
    {"id": 51, "rate": None, "model_dict": {"SF": SF_51, "Ealpha": Ealpha}},
    {"id": 52, "rate": None, "model_dict": {"SF": SF_52, "Ealpha": Ealpha}},
    {"id": 53, "rate": None, "model_dict": {"SF": SF_53, "Ealpha": Ealpha}},
    {"id": 54, "rate": None, "model_dict": {"SF": SF_54, "Ealpha": Ealpha}},
    {"id": 55, "rate": None, "model_dict": {"SF": SF_55, "Ealpha": Ealpha}},
    {"id": 56, "rate": None, "model_dict": {"SF": SF_56, "Ealpha": Ealpha}},
    {"id": 57, "rate": None, "model_dict": {"SF": SF_57, "Ealpha": Ealpha}},
    {"id": 58, "rate": None, "model_dict": {"SF": SF_58, "Ealpha": Ealpha}},
    {"id": 59, "rate": None, "model_dict": {"SF": SF_59, "Ealpha": Ealpha}},
    {"id": 60, "rate": None, "model_dict": {"SF": SF_60}},
    {"id": 61, "rate": None, "model_dict": {"SF": SF_61}},
    {"id": 62, "rate": None, "model_dict": {"SF": SF_62}}
    ]
    
    return dict_mod_vec

def loss_function(exp, teo, flag='numpy'):
    func = ((teo-exp)**2)/(exp**2)
    if flag == 'numpy':
        return np.mean(func)
    elif flag == 'torch':
        return torch.mean(func)
    else:
        raise ValueError(f"{flag} does not exist")


# 4. Instantiate and Run Optimizer
optimizer = opt.Optimizer(sim, 
                        lambda params: func_optimization(params, 'numpy'), 
                        lambda exp, teo: loss_function(exp, teo, 'numpy')
                        )

In [4]:


max_iters_tot = 1300
filename_results = "results/results_hier2_train_V4.h5"

pipeline = [
    ('decompose_stochastic', {'percent_info': 0.95, 'k_samples': 10}),
    ('optimize_stiff', {'max_iter': 70, 'bound_range': 1.2}),
    ('realign_sloppy', {}),
    ('optimize_sloppy', {'max_iter': 130}),
    ('check_convergence', {'tol_s': 1e-4})
]

# Instantiate
opt_hier = ha.HierarchicalOptimizer(optimizer, params_default_norm, pipeline=pipeline)
opt_hier.run(max_cycles=5, max_iter=max_iters_tot)


Starting Pipeline with 5 steps...

==================== CYCLE 1 ====================

[Step] Stochastic Decomposition (k=10)
loss:  783.7419346722897 iter:  2
loss:  783.7523754896821 iter:  3
loss:  783.7243705728511 iter:  4
loss:  783.8460035772019 iter:  5
loss:  783.7392360379195 iter:  6
loss:  783.7941344687899 iter:  7
loss:  783.6748347028587 iter:  8
loss:  783.7210654205633 iter:  9
loss:  783.707925675541 iter:  10
loss:  783.6656582620858 iter:  11
loss:  783.7209720743971 iter:  12
  -> Split: Stiff=1, Sloppy=4

[Step] Stiff Opt (Powell) Dim:1
loss:  783.7419346722897 iter:  13 params:  [0.]
loss:  4868.194830969617 iter:  14 params:  [-0.28328157]
loss:  3.908657013484796 iter:  15 params:  [0.28328157]
loss:  0.3829779162501245 iter:  16 params:  [0.63343685]
loss:  0.22822808680388132 iter:  17 params:  [0.45889739]
loss:  0.25267994303074204 iter:  18 params:  [0.53906093]
loss:  0.21610680947422878 iter:  19 params:  [0.47423078]
loss:  0.214620515178514 iter:  20 pa

loss:  0.1487683832802045 iter:  155 params:  [-0.00750229  0.14504173 -0.05237604  0.04776204]
loss:  0.11369472724302072 iter:  156 params:  [-0.00644021  0.1280625  -0.0438071   0.04049545]
loss:  0.16142478752530254 iter:  157 params:  [-0.00378838  0.1524779  -0.0502641   0.04733087]
loss:  0.163228540645042 iter:  158 params:  [-0.02328721  0.14506601 -0.05422087  0.0506476 ]
loss:  0.11160259495468944 iter:  159 params:  [-0.00488029  0.09939623 -0.03474666  0.03112199]
loss:  0.1422137637928344 iter:  160 params:  [-0.00581472  0.14054365 -0.04882326  0.04318936]
loss:  0.10417812297371738 iter:  161 params:  [-0.01078232  0.10433263 -0.03771892  0.03461368]
loss:  0.1314896151494428 iter:  162 params:  [ 0.00111371  0.10674037 -0.03597493  0.03359617]
loss:  0.23379454094999003 iter:  163 params:  [-0.01210046  0.11580104 -0.04185636  0.03772662]
loss:  0.1073017981381591 iter:  164 params:  [-0.00783131  0.10186443 -0.03623279  0.03286784]
loss:  0.1888449429510135 iter:  165

loss:  0.18272673369190753 iter:  167 params:  [-0.01364375  0.11157695 -0.04076788  0.03685856]
loss:  0.10422179619390425 iter:  168 params:  [-0.0107031   0.10515352 -0.03722641  0.03405643]
loss:  0.12371666388198564 iter:  169 params:  [-0.01286897  0.0952662  -0.03520999  0.03164369]
loss:  0.12846532030887198 iter:  170 params:  [-0.00744909  0.09173145 -0.03242617  0.02973226]
loss:  0.1180345527072272 iter:  171 params:  [-0.00899776  0.09669282 -0.0345116   0.03151384]
loss:  0.1368978562985791 iter:  172 params:  [-0.00628827  0.1087555  -0.03763487  0.0348822 ]
loss:  0.11443263174334461 iter:  173 params:  [-0.0112238   0.09863853 -0.03581621  0.03245332]
loss:  0.11488451952604145 iter:  174 params:  [-0.0112725   0.10830173 -0.03898556  0.0354818 ]
loss:  0.10419338984367843 iter:  175 params:  [-0.01070382  0.10539951 -0.03786707  0.03448981]
loss:  0.1468195881497339 iter:  176 params:  [-0.00878647  0.10973652 -0.03870639  0.03556056]
loss:  0.10805500571018073 iter: 

loss:  0.10317835079511171 iter:  201 params:  [-0.01555885  0.10587436 -0.04049533  0.0370478 ]


loss:  0.10264249554280021 iter:  202 params:  [-0.01552227  0.10624407 -0.04012109  0.03652086]
loss:  0.10254583560923526 iter:  203 params:  [-0.01729605  0.10670751 -0.04100802  0.03724213]


...

loss:  0.10204271747182264 iter:  292 params:  [-0.00251197  0.00353009  0.00609114 -0.00709863  0.00019689]


loss:  0.1019750269719502 iter:  293 params:  [-0.00241169  0.00361706  0.00666238 -0.00762334  0.00020234]


loss:  0.10199554024386323 iter:  294 params:  [-0.00228289  0.00373619  0.00569219 -0.00744433  0.00036537]
loss:  0.10197508091863292 iter:  295 params:  [-0.00324293  0.00516513  0.00821401 -0.01029893  0.00042034]


loss:  0.10190656345075805 iter:  296 params:  [-0.00229866  0.00437101  0.00629329 -0.00864499  0.00088547]


loss:  0.10181336474628126 iter:  297 params:  [-0.00194765  0.00468069  0.00618301 -0.0092792   0.00139308]
loss:  0.10181085890728933 iter:  298 params:  [-0.00287057  0.00522816  0.00843664 -0.01177381  0.00154527]
loss:  0.10167856120940158 iter:  299 params:  [-0.00316021  0.00613981  0.01021077 -0.01481806  0.00240088]
loss:  0.10171218843000027 iter:  300 params:  [-0.0021986   0.00495209  0.00822402 -0.01078356  0.0008456 ]
loss:  0.10164849085473923 iter:  301 params:  [-0.00290154  0.00608572  0.01010549 -0.0136769   0.00173952]
loss:  0.10148896879246055 iter:  302 params:  [-0.00321087  0.00726048  0.01231214 -0.01679319  0.0024266 ]
loss:  0.1015104819679479 iter:  303 params:  [-0.00192868  0.00549492  0.00922292 -0.01342     0.00248706]
loss:  0.10127389605914439 iter:  304 params:  [-0.00256671  0.00779414  0.01179876 -0.01841426  0.00361895]
loss:  0.10092397591446503 iter:  305 params:  [-0.00264423  0.00988268  0.01436695 -0.02380972  0.00532725]
loss:  0.10113145789

In [5]:
with h5py.File(filename_results, "w") as f:
    
    f.create_dataset("best_loss", data=opt_hier.history['best_loss'])
    f.create_dataset("iters", data=opt_hier.history['iters'])
    f.create_dataset("best_params", data=opt_hier.history['best_params'])
    
    f.create_dataset("best_params_end", data=opt_hier.phi)